In [13]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import json
import psutil
import time
import os

app_path = "../bin/daphne"
#args = "../test.daph"
args = "./normalize_matrix.daph n=5000"
args = os.path.abspath(args)
update_in_place = False

In [14]:
def run_command(update_in_place, args): 
    #command = "echo $$ && ../bin/daphne " + ("--update-in-place" if update_in_place else "") + " --timing " + args
    #command = ["../bin/daphne", (" --update-in-place" if update_in_place else ""), " --timing ", args]

    #split args into list
    args = args.split(" ")

    if update_in_place:
        command = ["../bin/daphne", "--update-in-place", "--timing"] + args
    else:
        command = ["../bin/daphne", "--timing"] + args

    print("Running command: " + ' '.join(command))    

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
   
    process_memory = psutil.Process(process.pid)
    current_pid = os.getpid()
    print("Current PID:", current_pid)

    # Print the process details
    print(f"Subprocess PID: {process.pid}")
    print(f"Subprocess name: {process_memory.name()}")
    print(f"Subprocess status: {process_memory.status()}")

    peak_mem = 0

    while process.poll() is None:  # Check if the process has finished
        mem_info = process_memory.memory_info().rss // 1024  # Memory usage in KB
        if mem_info > peak_mem:
            peak_mem = mem_info

        time.sleep(0.001)

    print(f"Peak memory usage: {peak_mem} KB")


    stdout, stderr = process.communicate()

    return command, peak_mem, stderr.decode().splitlines()[-1]

def run_benchmarks(update_in_place, args, n):
    results = []
    for i in range(n):
        print("\033[91m")
        print("Running benchmark " + str(i) + "...")
        print("\033[0m")
        results.append(run_command(update_in_place, args))
    print(results)
    return results

In [15]:
def benching(n_benchmarks, update_in_place, args):
    _results = run_benchmarks(update_in_place=update_in_place, args=args, n=n_benchmarks)

    title = _results[0][0]
    peak_mem = [result[1] for result in _results]

    results = [json.loads(json_str[2]) for json_str in _results]

    values_list = [[data[value_name] for data in results] for value_name in results[0].keys()]
    plt.figure(figsize=(10, 6))

    plt.boxplot(values_list, labels=results[0].keys())
    plt.title(title)
    plt.ylabel('Seconds')

    # Adjust layout
    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.show()

    print("Average peak memory:", np.mean(peak_mem), "KB")

benching(100, False, args)
benching(100, True, args)


Running benchmark 0...

Running command: ../bin/daphne --timing /home/niklas/daphne/bench/--vec ./normalize_matrix.daph n=5000
Current PID: 630075
Subprocess PID: 645507
Subprocess name: daphne
Subprocess status: running
Peak memory usage: 35456 KB


IndexError: list index out of range

In [ ]:
import time

pid = 2815730

while True:
    mem_info = psutil.Process(pid).memory_info()
    mem_usage = mem_info.rss / 1024 / 1024

    print(f"Memory usage: {mem_usage:.2f} MB")

    time.sleep(0.1)